---
title: "Aggregate Logistic Regression in PyMC vs Sklearn: A Step-by-Step Comparison"
format: html
---


In this tutorial, we will walk through the process of performing logistic regression using two different approaches: Scikit-learn (sklearn) and PyMC. The goal is to demonstrate how both methods work, compare their results, and show how you can present visualizations using Arviz for Bayesian modeling in PyMC.

We'll cover:

- **Data generation**: Creating a synthetic dataset for testing.
- **Modeling**: Running Logistic Regression with both Scikit-learn and PyMC.
- **Evaluation**: Evaluating and comparing model performance.
- **Visualizations**: Presenting results with useful plots using Arviz for PyMC.

# Core libraries

These core libraries are crucial for data analysis and modeling. Numpy handles numerical data, Pandas is for data manipulation, and Pymc supports probabilistic modeling. Arviz helps visualize Bayesian model results, while Matplotlib.pyplot is used for plotting. Sklearn offers tools for splitting data, applying machine learning models like logistic regression, and evaluating performance with metrics like accuracy, confusion matrix, and ROC curves.


In [ ]:
import numpy as np
import pandas as pd
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, auc

# Step 1: Data Generation

We'll start by generating synthetic data to test both models. The data consists of two features (X1, X2) and a target variable (Y), where Y is based on a logistic function of the features.


In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Generate random data
n_samples = 200
X1 = np.random.normal(loc=0, scale=1, size=n_samples)  # Feature 1
X2 = np.random.normal(loc=2, scale=1.5, size=n_samples)  # Feature 2

# Create a simple rule for the target variable
Y = (X1 + X2 + np.random.normal(scale=0.5, size=n_samples) > 2).astype(int)

# Create a DataFrame
df = pd.DataFrame({'X1': X1, 'X2': X2, 'Y': Y})

print(df.head())

In this case, we are creating two random features (X1, X2) and using a simple rule with noise to generate the binary target variable (Y). This is a typical setup for logistic regression problems.

# Step 2: Logistic Regression using Scikit-learn

Scikit-learn provides an easy way to perform logistic regression using the LogisticRegression class. First, we’ll split the data into training and testing sets, train the model, and evaluate its performance.


In [ ]:
X = df[['X1', 'X2']]
y = df['Y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Logistic Regression Model with Scikit-learn
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

# Make predictions
y_pred = log_reg.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred))

We evaluate the model using accuracy and a classification report that includes precision, recall, and F1-score.

## Visualize Scikit-learn Results


In [ ]:
# ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, log_reg.predict_proba(X_test)[:, 1])
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

This visual will allow us to assess the model’s performance in terms of its ability to distinguish between the two classes.


# Step 3: Logistic Regression using PyMC

Now, let's implement logistic regression using PyMC, a Bayesian modeling framework. We will build a probabilistic model that fits the data and then sample from the posterior distribution of the parameters.

# Your data generation code here (or loading from a file)

In [ ]:
np.random.seed(42)
X1 = np.random.normal(0, 1, 100)
X2 = np.random.normal(0, 1, 100)
y = (1 / (1 + np.exp(-(1 + 2 * X1 - 3 * X2)))) > 0.5
y = y.astype(int)

data = pd.DataFrame({'X1': X1, 'X2': X2, 'y': y})

def run_logistic_regression():
    # Define the model
    with pm.Model() as logistic_model:
        intercept = pm.Normal('intercept', mu=0, sigma=10)
        coef_X1 = pm.Normal('coef_X1', mu=0, sigma=10)
        coef_X2 = pm.Normal('coef_X2', mu=0, sigma=10)

        linear_model = intercept + coef_X1 * data['X1'] + coef_X2 * data['X2']

        p = pm.math.sigmoid(linear_model)
        likelihood = pm.Bernoulli('likelihood', p=p, observed=data['y'])

        trace = pm.sample(2000, return_inferencedata=True)

    # Traceplot and summary
    az.plot_trace(trace)
    print(az.summary(trace))

if __name__ == '__main__':
    run_logistic_regression()


## Visualizing Results in PyMC

Arviz provides a suite of functions to visualize the results from Bayesian models. After sampling, we can use az.plot_trace() to view the posterior distributions of the model parameters.


In [ ]:
az.plot_trace(trace)

This plot shows the posterior distributions for each parameter in the model, giving insights into their uncertainty and estimated values.

We can also use Arviz to visualize the posterior predictive checks or the model fit.

# Step 4: Comparing the Models

Now that we’ve trained and evaluated both models, let’s compare them. Here are some key differences:

- **Sklearn**: A frequentist approach, estimates fixed coefficients without incorporating uncertainty.
- **PyMC**: A Bayesian approach, where we model the uncertainty of the parameters and generate a posterior distribution for each.

## Model Comparison - Key Points:

- **Interpretation**: In PyMC, we interpret the posterior distributions of the parameters, while in Scikit-learn, we work directly with point estimates.
- **Uncertainty**: PyMC allows us to capture the uncertainty in model parameters, which is helpful in complex problems.
- **Evaluation**: Both models can be evaluated using metrics like accuracy, but PyMC also offers the advantage of uncertainty quantification through the posterior distributions.

## Final Visual Comparison (ROC)

Here, we can compare the ROC curves of both models:

- **Sklearn**: Generates a traditional ROC curve.
- **PyMC**: Since we have uncertainty in the model parameters, we can sample multiple predictions and visualize the spread.

# Conclusion

This walkthrough demonstrated how to implement logistic regression using Scikit-learn and PyMC. We compared both approaches, highlighting their key differences, and visualized results using standard metrics as well as Arviz for Bayesian modeling.

In Scikit-learn, we focused on efficiency and simplicity, while in PyMC, we explored the flexibility of Bayesian models and how they allow us to quantify uncertainty in the predictions.
